In [ ]:
"""
Created on Sat Oct 12 20:22:19 2019

@author: Fullah
"""
###################################################
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'        ## These two lines prevent an unnecessary message about the GPU.
###################################################
import warnings
warnings.filterwarnings('ignore') 
from __future__ import print_function 
from keras.models import Model
from keras.utils import np_utils
from keras.utils import plot_model
from keras.callbacks import ReduceLROnPlateau
from keras import optimizers
import numpy as np
import keras 
import pandas as pd
from keras.layers import Input
from keras.layers.convolutional import Conv1D
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers.recurrent import SimpleRNN
from keras.layers import concatenate
from keras_self_attention import SeqSelfAttention
from keras.layers import Dense
from keras.models import Model
import keras.layers as Layers
from keras import backend as K
%matplotlib inline

# fix random seed for reproducibility
np.random.seed(700013)

def readucr(filename):
    data = np.loadtxt(filename, delimiter = ',')
    Y = data[:,0]
    X = data[:,1:]
    return X, Y

# datalist = ['Adiac', 'Beef', 'CBF', 'ChlorineConcentration', 'CinC_ECG_torso', 'Coffee', 'Cricket_X', 'Cricket_Y', 'Cricket_Z', 
# 'DiatomSizeReduction', 'ECGFiveDays', 'FaceAll', 'FaceFour', 'FacesUCR', '50words', 'FISH', 'Gun_Point', 'Haptics', 
# 'InlineSkate', 'ItalyPowerDemand', 'Lighting2', 'Lighting7', 'MALLAT', 'MedicalImages', 'MoteStrain', 'NonInvasiveFatalECG_Thorax1', 
# 'NonInvasiveFatalECG_Thorax2', 'OliveOil', 'OSULeaf', 'SonyAIBORobotSurface', 'SonyAIBORobotSurfaceII', 'StarLightCurves', 'SwedishLeaf', 'Symbols', 
# 'synthetic_control', 'Trace', 'TwoLeadECG', 'Two_Patterns', 'uWaveGestureLibrary_X', 'uWaveGestureLibrary_Y', 'uWaveGestureLibrary_Z', 'wafer', 'WordsSynonyms', 'yoga']

# datalist  = ['Adiac', 'Beef', 'CBF', 'ChlorineConcentration', 'CinC_ECG_torso', 'Coffee']
datalist  = ['Adiac']
for each in datalist:
    fname = each
    trainX, trainY = readucr(fname+'/'+fname+'_TRAIN')
    testX, testY = readucr(fname+'/'+fname+'_TEST')
    nb_classes =len(np.unique(testY))
    trainY = (trainY - trainY.min())/(trainY.max()-trainY.min())*(nb_classes-1)
    testY = (testY - testY.min())/(testY.max()-testY.min())*(nb_classes-1)
#     batch_size = min(int(trainX.shape[0]/10), 16)
    batch_size = max(int(trainX.shape[0]/10), 64)

    trainX_mean = trainX.mean()
    trainX_std = trainX.std()
    trainX = (trainX - trainX_mean)/(trainX_std)
    testX = (testX - trainX_mean)/(trainX_std)

    trainX1 = trainX.reshape((trainX.shape[0], trainX.shape[1], 1))
    testX1 = testX.reshape((testX.shape[0], testX.shape[1], 1))
    #### This is the CCNN arm
    X2 = keras.layers.Input(trainX1.shape[1:]) 
    rnn1 = keras.layers.SimpleRNN(64, activation='relu', use_bias=True, kernel_initializer='glorot_uniform')(X2)
    rnn1 = keras.layers.normalization.BatchNormalization()(rnn1)
    rnn1 = keras.layers.Dropout(0.8)(rnn1)
    rnn1 = keras.layers.Reshape((64, 1))(rnn1) 
    X1 = keras.layers.Input(trainX1.shape[1:])
    conv1 = Conv1D(16, kernel_size=1, activation='relu', kernel_initializer='glorot_uniform', input_shape=(trainX1.shape[1:]))(X1)
    conv1 = keras.layers.normalization.BatchNormalization()(conv1)
    conv1 = keras.layers.Dropout(0.8)(conv1)
    conv1 = Dense(trainX1.shape[2], input_shape=(trainX1.shape[1],16))(conv1)
    conc1 = concatenate([conv1,rnn1], axis=-2, name = "contextual_convolutioal_layer")
    conv2 = Conv1D(8, kernel_size=1, activation='relu', kernel_initializer='glorot_uniform', name = "standard_cnn_layer")(conc1)
    conv2 = Dense(trainX1.shape[2], input_shape=(trainX1.shape[1],8))(conv2)
    conv2 = keras.layers.normalization.BatchNormalization()(conv2)
    conv2 = keras.layers.Dropout(0.7)(conv2)
    
    #### This is the CLSTM arm
    trainX2 = trainX.reshape([trainX.shape[0], trainX.shape[1]]) 
    testX2 = testX.reshape([testX.shape[0], testX.shape[1]])
    pd_trainX = pd.DataFrame(trainX2)
    pd_testX = pd.DataFrame(testX2)
    pd_testX = pd.DataFrame(testX2)
    roll_win1 = pd_trainX.rolling(window = 3).mean()  ### contextual feature(P)
    roll_win2 = pd_testX.rolling(window = 3).mean()
    roll_win1 = roll_win1.fillna(0)
    roll_win2 = roll_win2.fillna(0)
    trainX3 = keras.layers.np.concatenate((trainX2, roll_win1) , axis =1)
    testX3 = keras.layers.np.concatenate((testX2, roll_win2) , axis =1)
    trainX3 = K.variable(trainX3)
    testX3 = K.variable(testX3)
    trainX3 = Dense(trainX2.shape[1], input_shape=(trainX3.shape[1:]))(trainX3)
    testX3 = Dense(testX2.shape[1], input_shape=(testX3.shape[1:]))(testX3)
    trainX3 = K.eval(trainX3) 
    testX3 = K.eval(testX3)
    trainX4 = trainX3.reshape((trainX3.shape[0], trainX3.shape[1], 1))
    testX4 = testX3.reshape((testX3.shape[0], testX3.shape[1], 1))
    X3 = keras.layers.Input(trainX4.shape[1:])
    lstm11 = LSTM(8, return_sequences=False, kernel_initializer='glorot_uniform', activation='relu')(X3)
    lstm11 = keras.layers.Reshape((8, 1))(lstm11)
    lstm11 = keras.layers.Dropout(0.8)(lstm11)
    merge = keras.layers.concatenate([conv2, lstm11], axis =-2)
    
    avg = keras.layers.MaxPooling1D(pool_length=1, stride=None, border_mode='valid')(merge)
    avg = keras.layers.Dropout(0.6)(avg)
    att = SeqSelfAttention(attention_width=10,
          attention_activation='sigmoid',
          name='Attention',
                      )(avg)
    att = keras.layers.Dropout(0.5)(att)
    mlp1 = keras.layers.Dense(64, kernel_initializer='glorot_uniform', activation='relu')(att)
    mlp1 = keras.layers.Dropout(0.8)(mlp1)
    mlp2 = keras.layers.Dense(64, kernel_initializer='glorot_uniform', activation='relu')(mlp1)
    mlp2 = keras.layers.Dropout(0.8)(mlp2)
    flat5 = keras.layers.Flatten()(mlp2)
    output = keras.layers.Dense(nb_classes, activation='softmax')(flat5)
    model = keras.models.Model(inputs= [X1, X2, X3], outputs=output)
    Adam = optimizers.Adam(lr=0.5, beta_1=0.9, beta_2=0.999, epsilon=1e-7)
    model.compile(loss='sparse_categorical_crossentropy', optimizer= 'Adam', metrics=['accuracy'])
    reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor=0.7,
                      patience=50, min_lr=0.0001)
    hist = model.fit([trainX4,trainX1,trainX1],trainY, epochs=5, batch_size=batch_size, verbose=1,
                    validation_data=([testX4,testX1,testX1],testY), callbacks=[reduce_lr])
    ##### This print the testing results that has the minimum training errors.
    log = pd.DataFrame(hist.history)
    print(log.loc[log['loss'].idxmin]['loss'], log.loc[log['loss'].idxmin]['val_acc'])
#     model.summary()
    plot_model(model, to_file='confirm.pdf')